# Settings

In [1]:
from PyPDF2 import PdfFileMerger, PageRange, PdfFileWriter, PdfFileReader
import fitz
from os import listdir
import re

In [2]:
# Obtain a list of all the files in the 'output_data' folder
file_list = listdir('source_data/')
# Weed out those that are not PDFs
file_list = [file for file in file_list if re.search('pdf$', file)]

In [3]:
print(len(file_list))

70


# Bringing everybody together: Merge all PDF reports intro three different files

In [4]:
mega_pdf = PdfFileMerger() # Pulls together all files into one big one
pdf_first_pages = PdfFileMerger() # This one will contain all the first pages of each report
pdf_non_first_pages = PdfFileMerger() # This one will contain all the non-first pages of each report

for file in file_list:

    # Merge them all into one
    mega_pdf.append('source_data/' + file, pages=PageRange(':'))
    
    # File with only the first pages of all individual PDFs
    pdf_first_pages.append('source_data/' + file, pages=PageRange('0'))
    
    # File with the non-first pages of all individual PDFs
    pdf_non_first_pages.append('source_data/' + file, pages=PageRange('1:'))

mega_pdf.write('output_data/pdf/reports_all_pages.pdf')
pdf_first_pages.write('output_data/pdf/reports_first_pages.pdf')
pdf_non_first_pages.write('output_data/pdf/reports_non_first_pages.pdf')

mega_pdf.close()
pdf_first_pages.close()
pdf_non_first_pages.close()

# Cropping mayhem

**Warnign!**
For some reason, the PDF files produced by the next couple of code cells turns out to be corrupted and cannot be opened. If that happens, run them again. Usually the second or third time are successful. (Kindly do not ask for the reason of this bizarre phenomenon.)

## Non-first pages

In [8]:
ftz = fitz.open('output_data/pdf/reports_non_first_pages.pdf')
reader = PdfFileReader('output_data/pdf/reports_non_first_pages.pdf')
writer = PdfFileWriter()

# Consistency test
assert len(reader.pages) == len(ftz)

# (left, top, right, bottom) = (0.0, 0.0, 842.0, 595.0)
for i in range(0, len(reader.pages)):
    reader_page = reader.pages[i]
    fitz_page = ftz[i]
    
    right = fitz_page.searchFor('Update')[-1][0]
    
    reader_page.mediaBox.lowerLeft = (0, fitz_page.rect[3]-555) # (0, 595-555)
    reader_page.mediaBox.upperRight = (right*0.99, fitz_page.rect[3]-45) # (842-right, 595-45)
    writer.addPage(reader_page)

output_file = open('output_data/pdf/reports_non_first_pages_cropped.pdf', 'wb')
writer.write(output_file)

In [9]:
fitz_page.rect

Rect(0.0, 0.0, 842.0, 595.0)

## First pages

In [11]:
ftz = fitz.open('output_data/pdf/reports_first_pages.pdf')
reader = PdfFileReader('output_data/pdf/reports_first_pages.pdf')
writer = PdfFileWriter()

# Consistency test
assert len(reader.pages) == len(ftz)

# (left, top, right, bottom) = (0.0, 0.0, 842.0, 595.0)
for i in range(0, len(reader.pages)):
    reader_page = reader.pages[i]
    fitz_page = ftz[i]
    
    top = fitz_page.searchFor('Date/Time')[-1][3]
    right = fitz_page.searchFor('Update')[-1][0]
    
    reader_page.mediaBox.lowerLeft = (0, fitz_page.rect[3]-550) # (0, 595-550)
    reader_page.mediaBox.upperRight = (right*0.99, fitz_page.rect[3]-top) # (right, 595-top)
    writer.addPage(reader_page)

output_file = open('output_data/pdf/reports_first_pages_cropped.pdf', 'wb')
writer.write(output_file)